In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [18]:
r = requests.get('http://localhost:8000/products?format=json')
shop_df = pd.DataFrame(json.loads(r.content.decode()))
shop_df['name'] = [i.lower() for i in shop_df['name']]
shop_df.head()

In [66]:
filepath = 'shopping_lists/shoppingList13.json'
with open(filepath) as f:
    shopping_list = json.load(f)

In [92]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    to_drop = ['quick', 'canned', 'cooking', 'fat', 'ground', 'leaves', 'pasta',
               'plain', 'cooked', 'full-fat', 'pieces', 'scrambled', 'frozen', 'creamy']
    items_df['name'] = [[j for j in i if j not in to_drop] for i in items_df['name']]
    return items_df

In [93]:
items_df = get_items(shopping_list)

In [94]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,600627,[sprite],Beverages,14144,1.0,
1,600635,"[burrito, wrap]",Ethnic Foods,10118364,1.0,
2,600637,[tuna],Seafood,10015121,4.0,
3,600651,"[mat, beans]",Pasta and Rice,99144,1.0,
4,600655,[tomato],Produce,11529,1.0,
5,600659,[water],Pantry Items,14412,2.0,
6,600661,[chicken],Meat,5006,1.0,
7,600665,[fries],Frozen,11408,2.0,
8,600669,[bread],Bakery/Bread,18064,1.0,
9,600671,"[potato, chips]",Savory Snacks,19411,5.0,


In [95]:
matches_df = pd.DataFrame()
for product in items_df.iloc:
    regex = ''.join([f'(?=.*{i})' for i in product['name']])   
    entries = shop_df['name'].str.contains(regex)    
    if entries.any():
        matches = shop_df[entries]
        print(f"appending {' '.join(product['name'])}")
        matches_df = matches_df.append(matches)
    else:
        print(f"no such product: {' '.join(product['name'])}")

no such product: sprite
no such product: burrito wrap
appending tuna
appending mat beans
appending tomato
appending water
appending chicken
appending fries
appending bread
appending potato chips
appending tea
appending granola
no such product: m&m candy
appending yogurt
no such product: pizza mix


In [85]:
shop_df[shop_df['name'].str.contains('(?=.*)(?=.*pasta)')]

,store,name,img_url,price,currency,amount,unit
736,sainsburys,dolmio bolognese pasta sauce original 500g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.4,GBP,500g,unit
912,sainsburys,dolmio bolognese pasta sauce original 750g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.0,GBP,750g,unit


### add to the database:
    - ground cumin / cumin
    - hot sauce
    - almonds, agave
    - salsa
    - tahini
    - almond milk
    - worcestershire sauce
    - flax egg
    - maple syrup
    - lemon juice
    - garlic powder
    - gluten-free
    - lime juice
    - pesto
    - mozzarella
    - cornstarch
    - green bell pepper
    - red pepper flakes
    - pumpking seeds
    - dressings (caeasar etc)
    - cashews
    - dijon mustard
    - chili sauce
    - oats
    - peanut butter
    - tortilas
    - vanilla extract
    - starch
    - vinegar (white, apple cider)
    - soda, candy

### change

- convert 4x5 to 20 etc
- convert price to one common unit (pence)
- if ground and meat then mince

In [287]:
matches_df.sample(frac=0.2)

,store,name,img_url,price,currency,amount,unit
847,sainsburys,kitkat 4 finger milk chocolate bar multipack 4...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.00,GBP,4x41.5g,unit
720,sainsburys,cadbury dairy milk chocolate bar 200g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,200g,unit
723,sainsburys,cadbury dairy milk giant chocolate buttons bag...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,119g,unit
901,sainsburys,cadbury dairy milk chocolate bar 10x29.3g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.00,GBP,10x29.3g,unit
1089,sainsburys,"sainsbury's indulgence ice cream, milk chocola...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.35,GBP,4x110ml,unit
702,sainsburys,mcvitie's milk chocolate digestives biscuits 266g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.60,GBP,266g,unit
632,sainsburys,sainsbury's toastie thick sliced white bread 800g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,55.00,p,800g,unit


In [68]:
def match_product(name, price, quantity, unit):
    # TODO
    ...

In [97]:
import pytest

In [126]:
shop_df[['amount', 'unit']]

,amount,unit
0,Loose,kg
1,Loose,kg
2,,unit
3,Loose,kg
4,Loose,kg
...,...,...
1155,320g,unit
1156,650ml,unit
1157,610g,unit
1158,400g,unit


In [ ]:
@pytest.mark.parametrize('amount', [('4x41.5g', 166), ('10x29.3g', 293), ('4x110ml', 440), ('800g', 800), ('Loose', 1)])
def test_convert_amount(amount, unit):
    # TODO

In [ ]:
def convert_amount(amount):
    # TODO